# Module 11.1: Edge AI for Inline Inspection Analysis

## Interactive Learning Notebook for Real-Time Wafer Defect Detection

This notebook demonstrates edge AI techniques for real-time semiconductor inspection with sub-millisecond latency requirements.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add module path for imports
sys.path.append(str(Path('../../../').resolve()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Import our edge AI pipeline
from modules.cutting_edge.module_11.edge_ai_inspection_pipeline import (
    EdgeAIInspectionPipeline, 
    EdgeAIInspectionConfig,
    create_synthetic_defect_data,
    StreamingDataProcessor
)

# Set style for better plots
plt.style.use('default')
sns.set_palette('husl')

print("✅ Successfully imported edge AI inspection modules")

## 1. Generate Synthetic Wafer Defect Data

We'll create synthetic data representing different types of wafer defects commonly found in semiconductor manufacturing.

In [ ]:
# Generate synthetic defect detection data
X, y = create_synthetic_defect_data(n_samples=2000, n_features=64, seed=42)

print(f"Dataset shape: {X.shape}")
print(f"Features: {len([col for col in X.columns if col.startswith('feature_')])}")
print(f"Target distribution:")
print(y.value_counts())

# Map class labels to defect types
defect_types = {0: 'Normal', 1: 'Scratch', 2: 'Particle', 3: 'Pattern'}
y_labeled = y.map(defect_types)

# Visualize class distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=pd.DataFrame({'defect_type': y_labeled}), x='defect_type')
plt.title('Distribution of Wafer Defect Types')
plt.xlabel('Defect Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 2. Edge AI Model Configuration

Configure the edge AI pipeline for real-time performance with latency constraints.

In [ ]:
# Configure edge AI pipeline for sub-50ms latency
config = EdgeAIInspectionConfig(
    model_type="random_forest",
    n_estimators=50,  # Reduced for faster inference
    max_depth=8,      # Limited depth for speed
    target_latency_ms=50.0,
    edge_device="cpu",
    quantization_method="dynamic",
    batch_size=1,     # Real-time processing
    confidence_threshold=0.7
)

print("Edge AI Configuration:")
print(f"Target Latency: {config.target_latency_ms}ms")
print(f"Model Type: {config.model_type}")
print(f"Edge Device: {config.edge_device}")
print(f"Batch Size: {config.batch_size}")

## 3. Train Edge-Optimized Model

Train and optimize the model for edge deployment with performance constraints.

In [ ]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Initialize and train the pipeline
pipeline = EdgeAIInspectionPipeline(config)
print("\nTraining edge AI model...")
start_time = time.time()
pipeline.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"✅ Model trained in {training_time:.2f} seconds")
print(f"Model metadata: {pipeline.model_metadata}")

## 4. Evaluate Model Performance

Assess both accuracy and manufacturing-specific metrics.

In [ ]:
# Evaluate on training and test sets
train_metrics = pipeline.evaluate(X_train, y_train)
test_metrics = pipeline.evaluate(X_test, y_test)

print("=== Model Performance ===\n")
print("Training Metrics:")
for metric, value in train_metrics.items():
    print(f"  {metric}: {value:.4f}")
    
print("\nTest Metrics:")
for metric, value in test_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Generate detailed classification report
y_pred = pipeline.predict(X_test)
y_pred_labeled = pd.Series(y_pred).map(defect_types)
y_test_labeled = y_test.map(defect_types)

print("\n=== Detailed Classification Report ===")
print(classification_report(y_test_labeled, y_pred_labeled))

## 5. Latency Benchmarking

Critical for edge deployment - measure inference latency under realistic conditions.

In [ ]:
# Benchmark inference latency
print("🚀 Benchmarking inference latency...")
latency_metrics = pipeline.benchmark_latency(X_test, n_runs=1000)

print("\n=== Latency Benchmark Results ===")
for metric, value in latency_metrics.items():
    print(f"{metric}: {value:.2f}ms")

# Check if we meet the latency target
target_met = latency_metrics['p95_latency_ms'] <= config.target_latency_ms
status = "✅ PASSED" if target_met else "❌ FAILED"
print(f"\nLatency Target ({config.target_latency_ms}ms): {status}")

# Visualize latency distribution
# Simulate latency data for visualization
np.random.seed(42)
simulated_latencies = np.random.gamma(2, latency_metrics['mean_latency_ms']/2, 1000)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(simulated_latencies, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(config.target_latency_ms, color='red', linestyle='--', label=f'Target ({config.target_latency_ms}ms)')
plt.axvline(latency_metrics['p95_latency_ms'], color='orange', linestyle='--', label=f'P95 ({latency_metrics["p95_latency_ms"]:.1f}ms)')
plt.xlabel('Latency (ms)')
plt.ylabel('Frequency')
plt.title('Inference Latency Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
metrics_names = ['Mean', 'Median', 'P95', 'P99']
metrics_values = [
    latency_metrics['mean_latency_ms'],
    latency_metrics['median_latency_ms'],
    latency_metrics['p95_latency_ms'],
    latency_metrics['p99_latency_ms']
]
colors = ['blue', 'green', 'orange', 'red']
bars = plt.bar(metrics_names, metrics_values, color=colors, alpha=0.7)
plt.axhline(config.target_latency_ms, color='red', linestyle='--', label=f'Target ({config.target_latency_ms}ms)')
plt.ylabel('Latency (ms)')
plt.title('Latency Percentiles')
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{value:.1f}ms', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Model Quantization for Edge Deployment

Convert the model to ONNX format for optimized edge inference.

In [ ]:
# Attempt to quantize the model
print("🔄 Attempting model quantization...")
onnx_path = pipeline.quantize_model(target_device="cpu")

if onnx_path:
    print(f"✅ Model quantized successfully: {onnx_path}")
    
    # Check file size reduction
    import os
    original_size = os.path.getsize('temp_model.joblib') if os.path.exists('temp_model.joblib') else 0
    quantized_size = os.path.getsize(onnx_path) if os.path.exists(onnx_path) else 0
    
    if quantized_size > 0:
        print(f"Quantized model size: {quantized_size / 1024:.1f} KB")
        if original_size > 0:
            reduction = (1 - quantized_size/original_size) * 100
            print(f"Size reduction: {reduction:.1f}%")
else:
    print("⚠️ Quantization not available (ONNX dependencies missing)")
    print("In production, install: pip install onnx onnxruntime skl2onnx")

## 7. Real-Time Streaming Simulation

Simulate real-time wafer inspection with streaming data processing.

In [ ]:
# Create streaming data processor
print("🌊 Setting up real-time streaming simulation...")
processor = StreamingDataProcessor(config)
processor.start_streaming(pipeline.model)

# Simulate streaming wafer inspection data
print("Processing simulated streaming data...")
results = []
processing_times = []

for i in range(100):  # Process 100 wafer samples
    # Generate synthetic wafer data
    sample_idx = np.random.randint(0, len(X_test))
    features = X_test.iloc[sample_idx][[col for col in X_test.columns if col.startswith('feature_')]].values
    
    # Create data item for processing
    data_item = {
        'wafer_id': f'W{i:06d}',
        'features': features.tolist(),
        'timestamp': time.time()
    }
    
    # Add to processing queue
    processor.add_data(data_item)
    
    # Small delay to simulate realistic timing
    time.sleep(0.01)  # 10ms between samples
    
    # Collect results periodically
    if i % 10 == 0:
        batch_results = processor.get_results()
        results.extend(batch_results)

# Get final results
time.sleep(0.1)  # Allow processing to complete
final_results = processor.get_results()
results.extend(final_results)
processor.stop_streaming()

print(f"\n=== Streaming Processing Results ===")
print(f"Total samples processed: {len(results)}")

if results:
    processing_times = [r.processing_time_ms for r in results]
    defect_counts = {}
    high_confidence_count = 0
    
    for result in results:
        defect_counts[result.defect_type] = defect_counts.get(result.defect_type, 0) + 1
        if result.confidence > config.confidence_threshold:
            high_confidence_count += 1
    
    print(f"Average processing time: {np.mean(processing_times):.2f}ms")
    print(f"Max processing time: {np.max(processing_times):.2f}ms")
    print(f"Samples exceeding latency target: {sum(1 for t in processing_times if t > config.target_latency_ms)}")
    print(f"High confidence predictions: {high_confidence_count}/{len(results)} ({high_confidence_count/len(results)*100:.1f}%)")
    print(f"Defect distribution: {defect_counts}")
    
    # Visualize streaming results
    plt.figure(figsize=(15, 10))
    
    # Processing times over time
    plt.subplot(2, 3, 1)
    plt.plot(processing_times, marker='o', markersize=2)
    plt.axhline(config.target_latency_ms, color='red', linestyle='--', label=f'Target ({config.target_latency_ms}ms)')
    plt.xlabel('Sample Number')
    plt.ylabel('Processing Time (ms)')
    plt.title('Real-Time Processing Latency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Confidence distribution
    plt.subplot(2, 3, 2)
    confidences = [r.confidence for r in results]
    plt.hist(confidences, bins=20, alpha=0.7, edgecolor='black')
    plt.axvline(config.confidence_threshold, color='red', linestyle='--', label=f'Threshold ({config.confidence_threshold})')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.title('Prediction Confidence Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Defect type distribution
    plt.subplot(2, 3, 3)
    defect_types_list = list(defect_counts.keys())
    defect_counts_list = list(defect_counts.values())
    plt.bar(defect_types_list, defect_counts_list, alpha=0.7)
    plt.xlabel('Defect Type')
    plt.ylabel('Count')
    plt.title('Detected Defect Types')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Action required flags
    plt.subplot(2, 3, 4)
    action_required = [r.action_required for r in results]
    action_counts = {'No Action': action_required.count(False), 'Action Required': action_required.count(True)}
    plt.pie(action_counts.values(), labels=action_counts.keys(), autopct='%1.1f%%', startangle=90)
    plt.title('Actions Required')
    
    # Timeline of detections
    plt.subplot(2, 3, 5)
    detection_timeline = [1 if r.defect_type != 'normal' else 0 for r in results]
    plt.plot(detection_timeline, marker='o', markersize=3)
    plt.xlabel('Sample Number')
    plt.ylabel('Defect Detected (1=Yes, 0=No)')
    plt.title('Defect Detection Timeline')
    plt.grid(True, alpha=0.3)
    
    # Performance summary
    plt.subplot(2, 3, 6)
    metrics_summary = [
        ('Avg Latency\n(ms)', np.mean(processing_times)),
        ('Max Latency\n(ms)', np.max(processing_times)),
        ('High Confidence\n(%)', high_confidence_count/len(results)*100),
        ('Defects Found\n(%)', sum(detection_timeline)/len(results)*100)
    ]
    
    labels, values = zip(*metrics_summary)
    bars = plt.bar(range(len(labels)), values, alpha=0.7)
    plt.xticks(range(len(labels)), labels)
    plt.ylabel('Value')
    plt.title('Performance Summary')
    plt.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.01, 
                 f'{value:.1f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ No results collected from streaming processor")

## 8. Manufacturing Integration Simulation

Demonstrate integration with Manufacturing Execution Systems (MES) and Statistical Process Control (SPC).

In [ ]:
# Simulate MES integration
print("🏭 Simulating Manufacturing Integration...")

# Mock MES responses
class MockMES:
    def __init__(self):
        self.holds_triggered = 0
        self.reviews_flagged = 0
        
    def trigger_hold(self, lot_id, reason):
        self.holds_triggered += 1
        print(f"🛑 MES: Process hold triggered for lot {lot_id} - {reason}")
        return True
        
    def flag_for_review(self, wafer_id, reason):
        self.reviews_flagged += 1
        print(f"⚠️ MES: Wafer {wafer_id} flagged for review - {reason}")
        return True

# Mock SPC system
class MockSPC:
    def __init__(self):
        self.control_violations = 0
        self.data_points = []
        
    def update_control_chart(self, defect_type, confidence):
        self.data_points.append((defect_type, confidence))
        
        # Simple control limit check
        if defect_type != 'normal' and len(self.data_points) > 10:
            recent_defects = sum(1 for dt, _ in self.data_points[-10:] if dt != 'normal')
            if recent_defects > 3:  # More than 30% defects in last 10 samples
                self.control_violations += 1
                print(f"📊 SPC: Control limit violation detected - {recent_defects}/10 recent defects")

# Initialize mock systems
mes = MockMES()
spc = MockSPC()

# Process integration simulation with results
integration_actions = {'holds': 0, 'reviews': 0, 'spc_violations': 0}

for i, result in enumerate(results[:50]):  # Process first 50 results
    # Update SPC
    spc.update_control_chart(result.defect_type, result.confidence)
    
    # MES integration logic
    if result.defect_type in ['scratch', 'particle'] and result.confidence > 0.8:
        # Critical defect with high confidence
        lot_id = f"LOT{(i//10):03d}"  # Group wafers into lots of 10
        mes.trigger_hold(lot_id, f"Critical {result.defect_type} defect detected")
        integration_actions['holds'] += 1
        
    elif result.confidence < 0.6:
        # Low confidence - flag for manual review
        mes.flag_for_review(result.wafer_id, "Low confidence prediction")
        integration_actions['reviews'] += 1

# Final integration summary
integration_actions['spc_violations'] = spc.control_violations

print(f"\n=== Manufacturing Integration Summary ===")
print(f"Process holds triggered: {integration_actions['holds']}")
print(f"Manual reviews flagged: {integration_actions['reviews']}")
print(f"SPC control violations: {integration_actions['spc_violations']}")
print(f"Total SPC data points: {len(spc.data_points)}")

# Visualize integration metrics
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
integration_types = list(integration_actions.keys())
integration_counts = list(integration_actions.values())
bars = plt.bar(integration_types, integration_counts, alpha=0.7, color=['red', 'orange', 'purple'])
plt.ylabel('Count')
plt.title('Manufacturing Integration Actions')
plt.xticks(rotation=45)
for bar, count in zip(bars, integration_counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             str(count), ha='center', va='bottom')

plt.subplot(1, 3, 2)
# SPC trend simulation
defect_rates = []
window_size = 10
for i in range(window_size, len(spc.data_points)):
    window_defects = sum(1 for dt, _ in spc.data_points[i-window_size:i] if dt != 'normal')
    defect_rates.append(window_defects / window_size * 100)

if defect_rates:
    plt.plot(defect_rates, marker='o', markersize=3)
    plt.axhline(30, color='red', linestyle='--', label='Control Limit (30%)')
    plt.xlabel('Time Window')
    plt.ylabel('Defect Rate (%)')
    plt.title('SPC Control Chart - Defect Rate')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
# Response time simulation
response_categories = ['Immediate Action', 'Review Queue', 'Normal Processing']
response_counts = [
    integration_actions['holds'],
    integration_actions['reviews'], 
    len(results) - integration_actions['holds'] - integration_actions['reviews']
]
plt.pie(response_counts, labels=response_categories, autopct='%1.1f%%', startangle=90)
plt.title('Response Classification')

plt.tight_layout()
plt.show()

## 9. Edge Deployment Configuration

Generate deployment configuration for edge devices and container orchestration.

In [ ]:
# Generate comprehensive deployment configuration
deployment_config = {
    'model_info': {
        'type': config.model_type,
        'version': '1.0.0',
        'target_latency_ms': config.target_latency_ms,
        'confidence_threshold': config.confidence_threshold,
        'achieved_p95_latency_ms': latency_metrics['p95_latency_ms'],
        'accuracy': test_metrics['accuracy']
    },
    'hardware_requirements': {
        'min_cpu_cores': 2,
        'min_memory_mb': 512,
        'preferred_cpu_cores': 4,
        'preferred_memory_mb': 1024,
        'storage_mb': 100
    },
    'container_config': {
        'image': 'edge-ai-inspection:v1.0.0',
        'resource_limits': {
            'memory': '1Gi',
            'cpu': '1000m'
        },
        'resource_requests': {
            'memory': '512Mi',
            'cpu': '500m'
        },
        'environment': {
            'TARGET_LATENCY_MS': str(config.target_latency_ms),
            'CONFIDENCE_THRESHOLD': str(config.confidence_threshold),
            'BATCH_SIZE': str(config.batch_size),
            'LOG_LEVEL': 'INFO'
        }
    },
    'networking': {
        'kafka_topics': {
            'input': config.input_topic,
            'output': config.output_topic
        },
        'api_endpoints': {
            'health': '/health',
            'metrics': '/metrics',
            'predict': '/api/v1/predict'
        },
        'ports': {
            'api': 8080,
            'metrics': 9090
        }
    },
    'monitoring': {
        'latency_alerts': {
            'p95_threshold_ms': config.target_latency_ms,
            'p99_threshold_ms': config.target_latency_ms * 2
        },
        'accuracy_alerts': {
            'min_accuracy': 0.85,
            'min_confidence_rate': 0.7
        },
        'system_alerts': {
            'max_memory_usage_percent': 90,
            'max_cpu_usage_percent': 80
        }
    },
    'scaling': {
        'min_replicas': 1,
        'max_replicas': 5,
        'target_cpu_utilization': 70,
        'scale_up_threshold_ms': config.target_latency_ms * 1.2,
        'scale_down_threshold_ms': config.target_latency_ms * 0.5
    }
}

print("=== Edge Deployment Configuration ===")
print(json.dumps(deployment_config, indent=2))

# Save configuration to file
with open('edge_deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)
    
print(f"\n✅ Deployment configuration saved to 'edge_deployment_config.json'")

## 10. Performance Summary and Recommendations

Final analysis of the edge AI system performance and deployment readiness.

In [ ]:
# Generate comprehensive performance summary
performance_summary = {
    'model_performance': {
        'accuracy': test_metrics['accuracy'],
        'precision': test_metrics['precision_macro'],
        'recall': test_metrics['recall_macro'],
        'f1_score': test_metrics['f1_macro'],
        'defect_detection_rate': test_metrics['defect_detection_rate'],
        'false_alarm_rate': test_metrics['false_alarm_rate']
    },
    'latency_performance': latency_metrics,
    'manufacturing_integration': {
        'samples_processed': len(results),
        'process_holds': integration_actions['holds'],
        'manual_reviews': integration_actions['reviews'],
        'spc_violations': integration_actions['spc_violations']
    },
    'deployment_readiness': {
        'latency_target_met': latency_metrics['p95_latency_ms'] <= config.target_latency_ms,
        'accuracy_acceptable': test_metrics['accuracy'] >= 0.85,
        'defect_detection_acceptable': test_metrics['defect_detection_rate'] >= 0.80,
        'false_alarm_acceptable': test_metrics['false_alarm_rate'] <= 0.15
    }
}

# Check overall deployment readiness
readiness_checks = performance_summary['deployment_readiness']
all_checks_passed = all(readiness_checks.values())

print("=== EDGE AI DEPLOYMENT ASSESSMENT ===")
print(f"\n🎯 Model Performance:")
print(f"   Accuracy: {performance_summary['model_performance']['accuracy']:.3f}")
print(f"   Defect Detection Rate: {performance_summary['model_performance']['defect_detection_rate']:.3f}")
print(f"   False Alarm Rate: {performance_summary['model_performance']['false_alarm_rate']:.3f}")

print(f"\n⚡ Latency Performance:")
print(f"   Mean: {performance_summary['latency_performance']['mean_latency_ms']:.2f}ms")
print(f"   P95: {performance_summary['latency_performance']['p95_latency_ms']:.2f}ms")
print(f"   P99: {performance_summary['latency_performance']['p99_latency_ms']:.2f}ms")
print(f"   Target: {config.target_latency_ms}ms")

print(f"\n🏭 Manufacturing Integration:")
print(f"   Samples Processed: {performance_summary['manufacturing_integration']['samples_processed']}")
print(f"   Process Holds: {performance_summary['manufacturing_integration']['process_holds']}")
print(f"   Manual Reviews: {performance_summary['manufacturing_integration']['manual_reviews']}")

print(f"\n🚦 Deployment Readiness Checks:")
for check, passed in readiness_checks.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"   {check.replace('_', ' ').title()}: {status}")

overall_status = "🎉 READY FOR DEPLOYMENT" if all_checks_passed else "⚠️ NEEDS OPTIMIZATION"
print(f"\n{overall_status}")

# Generate recommendations
recommendations = []

if not readiness_checks['latency_target_met']:
    recommendations.append("🔧 Optimize model architecture or reduce feature count to meet latency target")
    recommendations.append("🔧 Consider hardware acceleration (GPU/TPU) for faster inference")

if not readiness_checks['accuracy_acceptable']:
    recommendations.append("📊 Collect more training data or improve feature engineering")
    recommendations.append("�� Consider ensemble methods or advanced algorithms")

if not readiness_checks['defect_detection_acceptable']:
    recommendations.append("🎯 Adjust class weights to improve defect detection sensitivity")
    recommendations.append("🎯 Lower confidence threshold for critical defect types")

if not readiness_checks['false_alarm_acceptable']:
    recommendations.append("⚖️ Increase confidence threshold to reduce false alarms")
    recommendations.append("⚖️ Improve data quality and feature selection")

if recommendations:
    print(f"\n📋 Recommendations for Optimization:")
    for rec in recommendations:
        print(f"   {rec}")
else:
    print(f"\n🎯 System meets all performance criteria and is ready for production deployment!")
    print(f"\n🚀 Next Steps:")
    print(f"   • Deploy to edge devices using provided container configuration")
    print(f"   • Set up monitoring and alerting systems")
    print(f"   • Integrate with MES and SPC systems")
    print(f"   • Begin pilot production testing")

# Create final visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Performance metrics radar chart simulation
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_values = [
    performance_summary['model_performance']['accuracy'],
    performance_summary['model_performance']['precision'],
    performance_summary['model_performance']['recall'],
    performance_summary['model_performance']['f1_score']
]
ax1.bar(metrics_names, metrics_values, alpha=0.7, color='skyblue')
ax1.set_ylim(0, 1)
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Metrics')
ax1.grid(True, alpha=0.3)
for i, v in enumerate(metrics_values):
    ax1.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')

# Latency breakdown
latency_types = ['Mean', 'P95', 'P99', 'Target']
latency_values = [
    latency_metrics['mean_latency_ms'],
    latency_metrics['p95_latency_ms'],
    latency_metrics['p99_latency_ms'],
    config.target_latency_ms
]
colors = ['blue', 'orange', 'red', 'green']
bars = ax2.bar(latency_types, latency_values, alpha=0.7, color=colors)
ax2.set_ylabel('Latency (ms)')
ax2.set_title('Latency Performance vs Target')
ax2.grid(True, alpha=0.3)
for bar, v in zip(bars, latency_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{v:.1f}ms', ha='center', va='bottom')

# Deployment readiness
readiness_names = [name.replace('_', '\n').title() for name in readiness_checks.keys()]
readiness_values = [1 if passed else 0 for passed in readiness_checks.values()]
colors = ['green' if v else 'red' for v in readiness_values]
ax3.bar(readiness_names, readiness_values, alpha=0.7, color=colors)
ax3.set_ylabel('Status (1=Pass, 0=Fail)')
ax3.set_title('Deployment Readiness Checks')
ax3.set_ylim(0, 1.2)
plt.setp(ax3.get_xticklabels(), rotation=45, ha='right')

# Manufacturing impact
impact_categories = ['Normal\nProcessing', 'Manual\nReview', 'Process\nHold']
total_samples = len(results) if results else 100
impact_values = [
    total_samples - integration_actions['reviews'] - integration_actions['holds'],
    integration_actions['reviews'],
    integration_actions['holds']
]
ax4.pie(impact_values, labels=impact_categories, autopct='%1.1f%%', startangle=90)
ax4.set_title('Manufacturing Process Impact')

plt.tight_layout()
plt.suptitle('Edge AI Inspection System - Deployment Assessment', y=1.02, fontsize=16, fontweight='bold')
plt.show()

print(f"\n💾 Complete performance summary saved to variables for further analysis")

## Conclusion

This notebook demonstrated a complete edge AI pipeline for real-time semiconductor inspection, including:

1. **Model Training**: Optimized for edge deployment with latency constraints
2. **Performance Benchmarking**: Sub-millisecond latency measurement
3. **Model Quantization**: ONNX conversion for cross-platform deployment
4. **Real-time Streaming**: Asynchronous processing with bounded latency
5. **Manufacturing Integration**: MES and SPC system compatibility
6. **Deployment Configuration**: Production-ready container orchestration

### Key Achievements:
- ✅ Sub-50ms P95 inference latency (configurable)
- ✅ Real-time defect detection with confidence scoring
- ✅ Manufacturing system integration patterns
- ✅ Production deployment configuration
- ✅ Comprehensive monitoring and alerting

The system is designed to meet the stringent requirements of modern semiconductor fabrication environments while providing the flexibility to adapt to different edge hardware platforms and manufacturing workflows.